Using python3

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import h5py
import numpy as np
import os,argparse,sys

The files are too big, so we extract jus the signal,

Their code from [utils/extract_sig_ref.py](https://github.com/haotianteng/Chiron/blob/master/chiron/utils/extract_sig_ref.py) from [chrion](https://github.com/haotianteng/Chiron)

In [60]:
def extract(root, output):
    count = 1
    root_folder = root
    out_folder = output
    if not os.path.isdir(root_folder):
        raise IOError('Input directory does not found.')
    if out_folder is None:
        raw_folder = os.path.abspath(os.path.join(out_folder,os.pardir,'raw'))
        ref_folder = os.path.abspath(os.path.join(out_folder,os.pardir,'reference'))
    else:
        if not os.path.isdir(FLAGS.output_dir):
            os.makedirs(FLAGS.output_dir)
        raw_folder = os.path.abspath(os.path.join(out_folder,'raw'))
        ref_folder = os.path.abspath(os.path.join(out_folder,'reference'))
    if not os.path.isdir(raw_folder):
        os.mkdir(raw_folder)
    if not os.path.isdir(ref_folder):
        os.mkdir(ref_folder)
    for file_n in os.listdir(root_folder):
        if file_n.endswith('fast5'):
            try:
                raw_signal,reference = extract_file(root_folder+os.path.sep+file_n)
                count+=1
                if len(raw_signal)==0:
                    print ("Failed in extracting "+(os.path.join(raw_folder,os.path.splitext(file_n)[0]+'.signal')))
                    continue
            except:
                continue
            signal_file = open(os.path.join(raw_folder,os.path.splitext(file_n)[0]+'.signal'),'w+')
            signal_file.write(" ".join([str(val) for val in raw_signal]))
            if len(reference)>0:
                ref_file = open(os.path.join(ref_folder,os.path.splitext(file_n)[0]+'_ref.fasta'),'w+')
                ref_file.write(reference)
        print("Extracted "+(os.path.join(raw_folder,os.path.splitext(file_n)[0]+'.signal')))

In [61]:
def extract_file(input_file):
    try:
        input_data = h5py.File(input_file, 'r')    
    except IOError:
        return False
    except:
        return False
    raw_attr = input_data['Raw/Reads/']
    read_name = list(raw_attr.keys())[0]
    #read_name = raw_attr.keys()[0]
    raw_signal = raw_attr[read_name+'/Signal'].value
    try:
        reference = input_data['Analyses/Basecall_1D_000/BaseCalled_template/Fastq'].value
        reference = ">template\n"+str(reference).split("\n")[0]
    except:
        try:
            reference = input_data['Analyses/Alignment_000/Aligned_template/Fasta'].value
        except:
            reference = ''
    return raw_signal,reference

Chiron was written in python2. trying to debug function using python3

In [64]:
inputfile="/data/nanopore/fast5Dir/f5.fast5"

In [65]:
input_data = h5py.File(inputfile, 'r')

In [66]:
raw_attr = input_data['Raw/Reads/']
read_name = list(raw_attr.keys())[0]
raw_signal = raw_attr[read_name+'/Signal'].value

In [67]:
read_name

'Read_919'

There's only 1 read

In [121]:
list(input_data['Raw/Reads/'].values())[0]

<HDF5 group "/Raw/Reads/Read_919" (1 members)>

In [68]:
raw_attr

<HDF5 group "/Raw/Reads" (1 members)>

In [69]:
raw_signal

array([1144,  612,  619, ...,  623,  617,  641], dtype=int16)

In [102]:
reference = input_data['Analyses/Basecall_1D_000/BaseCalled_template/Fastq'].value

In [103]:
type(reference)

numpy.bytes_

Trying to the base called sequence out, somehow chiron calls this the reference... really?

In [104]:
reference.decode('UTF-8').split('\n')[1]

'TGCCGCTGGTTCAGTTGTGCTCAGGTGGTCAACCCAAATGTCCATCAGATAAATGGTCAAACAGTGGGTATACAACAGGATTATCGGTCATAAACAGATAAAGCTCTGGTGCATGCAACAACATGAATTACAAACATTATGCCGTGAGATAAACCAGACACAAGGGACAACAAATATTGTATGATTCCATATGACCCAGGTCAAATAGGCAATTCATGGAGACAGAAAGTAGATCGAGTTTCCAGGCAAATAGGGAGATGGAGTTATTGCTAAATAAGGTACAGGTTTCTGTTTGGCCGTGATGGAAAAGTTTGAAATAGTGGTGATAAGTTTACAACATTGTGAATCAAAATTAATGTAATTGAATTGTATGTAGTGAGTTAAAATGAGTTTTATTATGTATATATCCACAATGAGATAAAGAATTTAAAATTTAAAAGTATTTTAAAATGTTAAGTGTATTTTAACATCATATTGTTGCATTCAAGCATAGGCTCATTCCCTCACAATTATTTGTGTGTGTTTGGGGTGTTTCCAGGCCTTTGATGTATATTGTGATAGTGATAGATATTAAAATTTTATAGCAATACGGTTAACATTTAAGAGAACTGGATTGTGGTAATATACCTATAATCCTGGTGAAGAACTAAGGTGGGAGGATGGCTTGAAGCCAGGTTCAGACCAGCCTGGGCAACATAGGAACACCATTTCTAAAATTAAAATTAATGAGTATATGTATATATTATATAATGCTGTAGCATTATAATATAATACAGGTCTGTATTCCTAATACTTAGAGCTGAATAGGAGATCGCATGGGCCAATTTTGAGTTACAGTGAGCTATGGTCATTGCCTGAACTCCATCCTGCGATGAGGTAGAACCCTCTCTTTTAATTCTTTTTAAAGAACAGATGTAAAGACCTTTAAATATAAAGGTAAAATAGGGGTGAAATGGAGGAAAAATATGCCATGCGCTGGTAACCAAAAAGCATGGCTAT

Save in fastA format

In [105]:
reference = ">template\n"+reference.decode('UTF-8').split('\n')[1]

In [106]:
reference

'>template\nTGCCGCTGGTTCAGTTGTGCTCAGGTGGTCAACCCAAATGTCCATCAGATAAATGGTCAAACAGTGGGTATACAACAGGATTATCGGTCATAAACAGATAAAGCTCTGGTGCATGCAACAACATGAATTACAAACATTATGCCGTGAGATAAACCAGACACAAGGGACAACAAATATTGTATGATTCCATATGACCCAGGTCAAATAGGCAATTCATGGAGACAGAAAGTAGATCGAGTTTCCAGGCAAATAGGGAGATGGAGTTATTGCTAAATAAGGTACAGGTTTCTGTTTGGCCGTGATGGAAAAGTTTGAAATAGTGGTGATAAGTTTACAACATTGTGAATCAAAATTAATGTAATTGAATTGTATGTAGTGAGTTAAAATGAGTTTTATTATGTATATATCCACAATGAGATAAAGAATTTAAAATTTAAAAGTATTTTAAAATGTTAAGTGTATTTTAACATCATATTGTTGCATTCAAGCATAGGCTCATTCCCTCACAATTATTTGTGTGTGTTTGGGGTGTTTCCAGGCCTTTGATGTATATTGTGATAGTGATAGATATTAAAATTTTATAGCAATACGGTTAACATTTAAGAGAACTGGATTGTGGTAATATACCTATAATCCTGGTGAAGAACTAAGGTGGGAGGATGGCTTGAAGCCAGGTTCAGACCAGCCTGGGCAACATAGGAACACCATTTCTAAAATTAAAATTAATGAGTATATGTATATATTATATAATGCTGTAGCATTATAATATAATACAGGTCTGTATTCCTAATACTTAGAGCTGAATAGGAGATCGCATGGGCCAATTTTGAGTTACAGTGAGCTATGGTCATTGCCTGAACTCCATCCTGCGATGAGGTAGAACCCTCTCTTTTAATTCTTTTTAAAGAACAGATGTAAAGACCTTTAAATATAAAGGTAAAATAGGGGTGAAATGGAGGAAAAATATGCCATGCGCTGGTAACCAAAA

In [107]:
ref_file = open("/data/nanopore/out/reference/test.fasta",'w+')
ref_file.write(reference)

12869

I'm lost, need to read this https://github.com/mw55309/PoreCamp/blob/master/PoreCamp.md

Files structure 
1. Analyses
    * Basecall_1D_000
        * BaseCalled_template
            * Fastq
    * Alignment_000
        * Aligned_template
            * Fasta
2. Raw
    * Reads
3. UniqueGlobalKey

So basically, I'm using the wrong file, I should really be looking at the raw.py file

In [19]:
os.chdir("/data/nanopore/data/rel3/${chr}/${fn}/")

In [20]:
directoryListing = os.listdir()

In [24]:
directoryListing[:10]

['DEAMERNANOPORE_20161117_FNFAB43577_MN16450_sequencing_run_MA_821_R9_4_NA12878_11_17_16_88738_ch279_read682_strand.fast5',
 'makeson_PC_20161117_FNFAB44989_MN17284_sequencing_run_MA_470_R9_4_NA12878_11_17_16_64834_ch261_read1876_strand.fast5',
 'MinION2_20160921_FNFAB39075_MN16454_sequencing_run_Chip97_Human_R9_4_tune_12329_ch184_read55_strand.fast5',
 'fmh_15l4526_20161102_FNFAB42828_MN16457_sequencing_run_161101_Human_genomic_LSK108R9_4_32164_ch266_read2272_strand1.fast5',
 'MinION2_20160921_FNFAB39075_MN16454_sequencing_run_Chip97_Human_R9_4_tuned_89079_ch206_read120_strand.fast5',
 'LomanLabz_PC_20160923_FNFAB39043_MN17250_sequencing_run_Human_1D_ligation_R9_4_61260_ch62_read374_strand.fast5',
 'PLSP57501_20161014_FNFAB42205_MN16458_sequencing_run_Hum94_jt_97011_ch218_read341_strand1.fast5',
 'PLSP61583_20160922_FNFAB390088_MN17048_sequencing_run_Hum_94_2_72021_ch115_read4_strand.fast5',
 'LomanLabz_PC_20161117_FNFAB46683_MN17024_sequencing_run_20161117_Human_R9_4_Rapid_72651_ch50

In [35]:
% pwd

'/data/nanopore/data/rel3/${chr}/${fn}'

In [36]:
os.chdir("/data/nanopore/")

In [37]:
inputDir = "/data/nanopore/fast5Dir/"
outputDir = "/data/nanopore/out/"

In [47]:
signal, ref = extract_file("/data/nanopore/fast5Dir/f5.fast5")

In [39]:
signal

array([1144,  612,  619, ...,  623,  617,  641], dtype=int16)

In [48]:
ref

'>template\nb\'@de5294ff-20eb-463e-bebb-bb79c4ca4f9d_Basecall_Alignment_template DEAMERNANOPORE_20161117_FNFAB43577_MN16450_sequencing_run_MA_821_R9_4_NA12878_11_17_16_88738_ch162_read919_strand\\nTGCCGCTGGTTCAGTTGTGCTCAGGTGGTCAACCCAAATGTCCATCAGATAAATGGTCAAACAGTGGGTATACAACAGGATTATCGGTCATAAACAGATAAAGCTCTGGTGCATGCAACAACATGAATTACAAACATTATGCCGTGAGATAAACCAGACACAAGGGACAACAAATATTGTATGATTCCATATGACCCAGGTCAAATAGGCAATTCATGGAGACAGAAAGTAGATCGAGTTTCCAGGCAAATAGGGAGATGGAGTTATTGCTAAATAAGGTACAGGTTTCTGTTTGGCCGTGATGGAAAAGTTTGAAATAGTGGTGATAAGTTTACAACATTGTGAATCAAAATTAATGTAATTGAATTGTATGTAGTGAGTTAAAATGAGTTTTATTATGTATATATCCACAATGAGATAAAGAATTTAAAATTTAAAAGTATTTTAAAATGTTAAGTGTATTTTAACATCATATTGTTGCATTCAAGCATAGGCTCATTCCCTCACAATTATTTGTGTGTGTTTGGGGTGTTTCCAGGCCTTTGATGTATATTGTGATAGTGATAGATATTAAAATTTTATAGCAATACGGTTAACATTTAAGAGAACTGGATTGTGGTAATATACCTATAATCCTGGTGAAGAACTAAGGTGGGAGGATGGCTTGAAGCCAGGTTCAGACCAGCCTGGGCAACATAGGAACACCATTTCTAAAATTAAAATTAATGAGTATATGTATATATTATATAATGCTGTAGCATTATAATATAATACAGGTCTGTATTCCTAATACTTAGAGCTGA

Weird they extracted the reference (which is the output from another HMM or RNN), next part will be using the raw.py

Trying out raw.py

In [3]:
os.chdir("/data/nanopore/script/")

need to import his own class labelop

In [4]:
import labelop

In [5]:
def extract_file(input_file,output_file, basecall_group, basecall_subgroup):
    try:
        (raw_data, raw_label, raw_start, raw_length) = labelop.get_label_raw(input_file,basecall_group,basecall_subgroup)
    except IOError:
        return False
    except:
        return False
    f_signal = open(output_file+'.signal','w+')
    f_label = open(output_file+'.label','w+')
    f_signal.write(" ".join(str(val) for val in raw_data))
    for index,start in enumerate(raw_start):    
        f_label.write("%d %d %c\n"%(start,start+raw_length[index],str(raw_label['base'][index])))
    f_signal.close()
    f_label.close()
    return True

In [114]:
basecall_group = "Basecall_1D_000"
basecall_subgroup = "BaseCalled_template"

In [116]:
extract_file("/data/nanopore/fast5Dir/f5.fast5", "/data/nanopore/out/signalFile", "Basecall_1D_000", "BaseCalled_template")

False

In [5]:
(raw_data, raw_label, raw_start, raw_length) = labelop.get_label_raw("/data/nanopore/fast5Dir/f5.fast5", "Basecall_1D_000", "BaseCalled_template")

RuntimeError: Corrected data know found.

Error above cause the fast5 file hasnt been resquiggled yet... 